In [92]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.vgg19 import decode_predictions
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras as kf
import cv2
import numpy as np

In [61]:
# creating data generator for training set
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# creating data generator for test set
test_datagen = ImageDataGenerator(rescale=1./255)

In [62]:
# loading the training data after required modifications
train_data = train_datagen.flow_from_directory(
        './datasets/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        color_mode = 'rgb')

# loading validation data after required modifications
validation_data = test_datagen.flow_from_directory(
        './datasets/validation',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        color_mode = 'rgb')

Found 267 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [63]:
# making the convolutional model iusing VGG19 architecture
conv_model = tf.keras.applications.VGG19(
    include_top= False,
    weights="imagenet",
    input_tensor=None,
    input_shape= [224, 224] + [3],
    pooling=None,
)

In [64]:
# making classifier

# flattening the output of the convolutional part
x = kf.layers.Flatten()(conv_model.output)

# three hidden layers
x = kf.layers.Dense(100, activation='relu')(x)
x = kf.layers.Dense(100, activation='relu')(x)
x = kf.layers.Dense(100, activation='relu')(x)

# output layer
predictions = kf.layers.Dense(3, activation='softmax')(x)

In [65]:
# attaching both the convolutional model and the classification model
full_model = kf.models.Model(inputs=conv_model.input, outputs=predictions)

In [66]:
# printing the summary of the full model (convolutional model and classification model)
full_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [67]:
# making CNN training false
for layer in conv_model.layers:
  layer.trainable = False

In [68]:
# complling the model
full_model.compile(loss='categorical_crossentropy',
                  optimizer=kf.optimizers.Adamax(lr=0.001),
                  metrics=['accuracy'])

In [69]:
# fitting data into the final model using .fit_generator function
# .fit_generator function is being used for handling huge data
results = full_model.fit_generator(
  train_data,
  validation_data= validation_data,
  epochs=5,
  steps_per_epoch=len(train_data),
  validation_steps=len(test_data)
)

Epoch 1/5
9/9 [==============================] - 102s 11s/step - loss: 1.0960 - accuracy: 0.4510 - val_loss: 1.0255 - val_accuracy: 0.3500
Epoch 2/5
9/9 [==============================] - 82s 9s/step - loss: 0.5022 - accuracy: 0.8305
Epoch 3/5
9/9 [==============================] - 83s 9s/step - loss: 0.2402 - accuracy: 0.9226
Epoch 4/5
9/9 [==============================] - 83s 9s/step - loss: 0.1539 - accuracy: 0.9921
Epoch 5/5
9/9 [==============================] - 83s 9s/step - loss: 0.1168 - accuracy: 0.9796


In [70]:
# saving the trained model
# accuracy 97.96 %
filepath = './model_trained/model_1.h5'
full_model.save(filepath)